In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import datetime as dt
import os
import time

In [3]:
plt.style.use('dark_background')
mpl.rcParams['figure.figsize'] = (12, 6)

In [4]:
raw_folder = '/home/sid/mystuff/myprogs/flirt/projects/product_analytics/customer_segmentation/data/raw'
datapath = os.path.join(raw_folder, 'online_retail.xlsx')

In [5]:
df = pd.read_excel(datapath, parse_dates=['InvoiceDate'], engine='openpyxl')

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df['InvoiceDate'] = df['InvoiceDate'].dt.date

In [9]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
320616,565056,23297,SET 40 HEART SHAPE PETIT FOUR CASES,16,2011-08-31,1.65,16278.0,United Kingdom
312849,564375,22570,FELTCRAFT CUSHION RABBIT,4,2011-08-25,3.75,13078.0,United Kingdom
333031,566164,21238,RED RETROSPOT CUP,3,2011-09-09,1.63,NaN,United Kingdom
6319,536876,21892,TRADITIONAL WOODEN CATCH CUP GAME,1,2010-12-03,2.51,NaN,United Kingdom
216800,555854,21191,LARGE WHITE HONEYCOMB PAPER BELL,2,2011-06-07,2.10,12748.0,United Kingdom


In [10]:
df = df[pd.notnull(df['CustomerID'])]
df = df[(df['Quantity'] > 0)]
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14117,537462,84997A,GREEN 3 PIECE POLKADOT CUTLERY SET,4,2010-12-07,3.75,17228.0,United Kingdom
77372,542720,22644,CERAMIC CHERRY CAKE MONEY BANK,12,2011-01-31,1.45,12778.0,Netherlands
386813,570247,23542,70'S ALPHABET WALL ART,2,2011-10-10,8.25,15193.0,United Kingdom
255694,559362,21576,LETS GO SHOPPING COTTON TOTE BAG,6,2011-07-08,2.25,12635.0,Germany
201785,554323,22728,ALARM CLOCK BAKELIKE PINK,4,2011-05-23,3.75,16729.0,United Kingdom


In [11]:
df.shape

(397924, 8)

In [12]:
df['Sales'] = df['Quantity'] * df['UnitPrice'] ## similar to Revenue

In [13]:
cols_of_interest = ['CustomerID', 'InvoiceDate', 'Sales']

df = df[cols_of_interest]

In [14]:
df['CustomerID'].nunique()

4339

In [15]:
df.sample(5)

,CustomerID,InvoiceDate,Sales
381137,13422.0,2011-10-06,23.80
304461,16706.0,2011-08-17,6.24
90389,13893.0,2011-02-15,27.00
376090,13317.0,2011-10-04,17.00
392945,13985.0,2011-10-12,15.30
